In [2]:
%matplotlib notebook
#%matplotlib inline

import copy
import numpy as np
import random
import matplotlib.pyplot as plt
from _converter_v2 import Event2Dict
from _evaluation import print_metrics

In [22]:
import json
from collections import deque

prod_id = "ABU4"

path = "../offline/"

with open(path+"layouts/"+prod_id+".json") as j:    
    layout = json.load(j)

files = [prod_id+".txt"]
data = []
bad = 0
for filename in files:
    print("Loading rows... {}".format(filename))
    with open(path+filename) as f:
        for line in f:
            try:
                features = Event2Dict(json.loads(line), layout)
                features = features.values()
    
                data.append(features)
                print(np.asarray(data).shape)
#                 print(type(data), type(features))
            except Exception, e:
                print(e)
                break
                bad+=1

print("Errors: {}".format(bad))
print("Loaded: {}".format(len(data)))

""" random split seed """
data = np.asarray(data)
print(data.shape)
numerics = data[:,2:-1]
# for n in numerics[data[:][-1]=='true']:
#     plt.plot(n);
mask = np.random.rand(len(data)) < 0.9
# print(mask)

Loading rows... ABU4.txt
(1L, 44L)
(2L,)
(3L,)
(4L,)
(5L,)
(6L,)
(7L,)
(8L,)
(9L,)
(10L,)
(11L,)
(12L,)
(13L,)
(14L,)
(15L,)
(16L,)
(17L,)
(18L,)
(19L,)
(20L,)
(21L,)
(22L,)
(23L,)
(24L,)
(25L,)
(26L,)
(27L,)
(28L,)
(29L,)
(30L,)
(31L,)
(32L,)
(33L,)
(34L,)
(35L,)
(36L,)
(37L,)
(38L,)
(39L,)
(40L,)
(41L,)
(42L,)
(43L,)
(44L,)
(45L,)
(46L,)
(47L,)
(48L,)
(49L,)
(50L,)
(51L,)
(52L,)
(53L,)
(54L,)
(55L,)
(56L,)
(57L,)
(58L,)
(59L,)
(60L,)
(61L,)
(62L,)
(63L,)
(64L,)
(65L,)
(66L,)
(67L,)
(68L,)
(69L,)
(70L,)
(71L,)
(72L,)
(73L,)
(74L,)
(75L,)
(76L,)
(77L,)
(78L,)
(79L,)
(80L,)
(81L,)
(82L,)
(83L,)
(84L,)
(85L,)
(86L,)
(87L,)
(88L,)
(89L,)
(90L,)
(91L,)
(92L,)
(93L,)
(94L,)
(95L,)
(96L,)
(97L,)
(98L,)
(99L,)
(100L,)
(101L,)
(102L,)
(103L,)
(104L,)
(105L,)
(106L,)
(107L,)
(108L,)
(109L,)
(110L,)
(111L,)
(112L,)
(113L,)
(114L,)
(115L,)
(116L,)
(117L,)
(118L,)
(119L,)
(120L,)
(121L,)
(122L,)
(123L,)
(124L,)
(125L,)
(126L,)
(127L,)
(128L,)
(129L,)
(130L,)
(131L,)
(132L,)
(133L,)
(134L,)
(135L,)

KeyboardInterrupt: 

In [18]:
a = []
a.append(['1',2])
a.append([3,4])
x = [[1, 2], [3, 4]] 
y = np.array(x)
print(type(a), type(x), type(y), y.shape, np.asarray(a).shape)

(<type 'list'>, <type 'list'>, <type 'numpy.ndarray'>, (2L, 2L), (2L, 2L))


In [6]:
train = data[mask]
test = data[~mask]
print(test[0:2][1:10])
print("Train Total: {} Good: {} Faulty: {} Ratio: {}".format(len(train), len(train[train[:,-1]=='True']), len(train[train[:,-1]=='False']), float(len(train[train[:,-1]=='False']))/len(train)))
print("Test  Total: {} Good: {} Faulty: {} Ratio: {}".format(len(test), len(test[test[:,-1]=='True']), len(test[test[:,-1]=='False']), float(len(test[test[:,-1]=='False']))/len(train)))

[ array([u'SMTLine/B260/P912/', u'ABU4', u'100', u'199.89', u'43.18',
       u'32.23', u'20.93', u'43.03', u'54.97', u'23', u'77.95', u'65.86',
       u'31.98', u'119.96', u'119.8', u'599.6', u'600.08', u'43.22',
       u'32.03', u'20.99', u'42.94', u'55.04', u'22.95', u'78.04',
       u'65.81', u'31.92', u'104.21', u'42.92', u'32.04', u'21.04',
       u'42.98', u'55.09', u'22.89', u'77.83', u'66.01', u'31.71', u'9.85',
       u'15.06', u'10.26', u'6.95', u'5.9', u'201.67', u'true'], 
      dtype='<U18')]


IndexError: too many indices for array

In [ ]:
faulty = train[train[:,-1]=='False']
not_faulty = train[train[:,-1]=='True']
# train = np.concatenate((not_faulty[:len(faulty)*5], faulty))
# samples = np.random.choice(len(not_faulty), 5000, replace=False)
# train = np.concatenate((not_faulty[samples], faulty))
# train = np.random.permutation(train)
print("Train Total: {} Good: {} Faulty: {} Ratio: {}".format(len(train), len(train[train[:,-1]=='True']), len(train[train[:,-1]=='False']), float(len(train[train[:,-1]=='False']))/len(train)))

train_data = train[:,2:-1].astype(np.float32)
test_data = test[:,2:-1].astype(np.float32)
"""
Quality_OK is mapped to Faultiness
    'False' -> 1 (Faulty)
    'True'  -> 0 (Good)
"""
train_labels = np.array(train[:,-1]=='False').astype(np.int32)
test_labels = np.array(test[:,-1]=='False').astype(np.int32)

In [ ]:
""" Random Forest """
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,  class_weight={0:1.0, 1:0.05}, max_depth=90, n_jobs=4)

%time clf = clf.fit(train_data, train_labels)
print_metrics(train_labels, clf.predict(train_data))
print_metrics(test_labels, clf.predict(test_data))

In [ ]:
""" Gradient Boosting """
from sklearn.ensemble import GradientBoostingClassifier

losses = ['deviance', 'exponential']
clf = GradientBoostingClassifier(loss=losses[0], n_estimators=100,  max_depth=15, learning_rate=0.1)

%time clf = clf.fit(train_data, train_labels)
print_metrics(train_labels, clf.predict(train_data))
print_metrics(test_labels, clf.predict(test_data))

In [ ]:
""" Random Forest """
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics


clf = RandomForestClassifier(n_estimators=100,  class_weight={0:0.98, 1:0.02}, max_depth=90, n_jobs=4)
# clf = DecisionTreeClassifier(criterion="gini", random_state=1, max_depth=100,  class_weight={0:1.0, 1:0.05})

MCCs = []
HLs = []
ratio = 0.98/0.02
for tr in xrange(10,1720,100):
    small = np.concatenate((not_faulty[:tr*ratio], faulty[:tr]))
    small_data = small[:,2:-1].astype(np.float32)
    small_labels = np.array(small[:,-1]=='False').astype(np.int32)
    clf = clf.fit(small_data, small_labels)
    mcc = metrics.matthews_corrcoef(test_labels, clf.predict(test_data))
    hl = metrics.hamming_loss(test_labels, clf.predict(test_data))
    print("{}\t{}\t{}".format(tr, mcc, hl))
    MCCs.append(mcc)
    HLs.append(hl)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

clf = GradientBoostingClassifier(loss='exponential', n_estimators=100,  max_depth=90, learning_rate=0.1, subsample=1.0)

GBMCCs = []
GBHLs = []
for tr in xrange(10,1720,100):
    small = np.concatenate((not_faulty[:tr*10], faulty[:tr]))
    small_data = small[:,2:-1].astype(np.float32)
    small_labels = np.array(small[:,-1]=='False').astype(np.int32)
    clf = clf.fit(small_data, small_labels)
    mcc = metrics.matthews_corrcoef(test_labels, clf.predict(test_data))
    hl = metrics.hamming_loss(test_labels, clf.predict(test_data))
    print("{}\t{}\t{}".format(tr, mcc, hl))
    GBMCCs.append(mcc)
    GBHLs.append(hl)

In [ ]:
rrange = xrange(10,1720,100)
fig = plt.figure()

ax1 = fig.add_subplot(111)
ax1.set_ylabel('Matthew\'s Correlation Coefficient (MCC)')
ax1.set_xlabel("Faulties")
p = ax1.plot(rrange,MCCs, label="Random Forest")
ax1.axhline(y=0.9, color='g', linestyle='dashed')
ax1.grid(True)
ax1.legend(loc=4)
# for xy in zip(rrange, MCCs):  
#     ax1.annotate("%.2f"%xy[1], xy=xy, textcoords='data')

ax3 = fig.add_subplot(111)
ax3.plot(rrange,GBMCCs, label="Gradient Boosting")
ax3.legend(loc=4)

ax2 = ax1.twiny()
ax2.cla()
ax2.set_xlabel('Data points')
ax2.plot(np.array(rrange)*49,np.ones(18))

plt.show()

In [ ]:
rrange = xrange(10,1720,100)
fig = plt.figure()

ax1 = fig.add_subplot(111)
ax1.set_ylabel('Hamming Loss')
ax1.set_xlabel("Faulties")
# ax1.plot(rrange,MCCs)
# ax1.axhline(y=0.9, color='g', linestyle='dotted')
ax1.grid(True)
ax1.fill_between(rrange,HLs, alpha=0.2, color='b', label="Random Forest")
ax1.set_autoscaley_on(False)
ax1.set_ylim([0.0,0.1])

ax3 = fig.add_subplot(111)
ax3.fill_between(rrange,GBHLs, alpha=0.5, color='g', label="Gradient Boosting")
ax3.legend(loc=1)

ax2 = ax1.twiny()
ax2.cla()
ax2.set_xlabel('Data points')
ax2.plot(np.array(rrange)*49,np.zeros(18))

plt.show()

In [ ]:
""" Random Forest """
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics


clf = RandomForestClassifier(n_estimators=100,  class_weight={0:0.98, 1:0.02}, max_depth=90, n_jobs=4)
# clf = DecisionTreeClassifier(criterion="gini", random_state=1, max_depth=100,  class_weight={0:1.0, 1:0.05})

faulty = train[train[:,-1]=='False']
not_faulty = train[train[:,-1]=='True']

sMCCs = []
sHLs = []
ratio = 0.98/0.02
for tr in xrange(10,1720,100):
    small = np.concatenate((not_faulty[:], faulty[:tr]))
    small_data = small[:,2:-1].astype(np.float32)
    small_labels = np.array(small[:,-1]=='False').astype(np.int32)
    clf = clf.fit(small_data, small_labels)
    mcc = metrics.matthews_corrcoef(test_labels, clf.predict(test_data))
    hl = metrics.hamming_loss(test_labels, clf.predict(test_data))
    print("{}\t{}\t{}".format(tr, mcc, hl))
    sMCCs.append(mcc)
    sHLs.append(hl)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

clf = GradientBoostingClassifier(loss='exponential', n_estimators=100,  max_depth=90, learning_rate=0.1, subsample=1.0)

sGBMCCs = []
sGBHLs = []
for tr in xrange(10,1720,100):
    small = np.concatenate((not_faulty[:], faulty[:tr]))
    small_data = small[:,2:-1].astype(np.float32)
    small_labels = np.array(small[:,-1]=='False').astype(np.int32)
    clf = clf.fit(small_data, small_labels)
    mcc = metrics.matthews_corrcoef(test_labels, clf.predict(test_data))
    hl = metrics.hamming_loss(test_labels, clf.predict(test_data))
    print("{}\t{}\t{}".format(tr, mcc, hl))
    sGBMCCs.append(mcc)
    sGBHLs.append(hl)

In [ ]:
rrange = xrange(10,1720,100)
fig = plt.figure()

ax1 = fig.add_subplot(111)
ax1.set_ylabel('Matthew\'s Correlation Coefficient (MCC)')
# ax1.set_xlabel('Positive samples (faulties)')
ax1.set_xlabel("Faulties")
p = ax1.plot(rrange,sMCCs, label="Random Forest")
ax1.axhline(y=0.9, color='g', linestyle='dashed')
ax1.grid(True)
ax1.legend(loc=4)

ax3 = fig.add_subplot(111)
ax3.plot(rrange,sGBMCCs, label="Gradient Boosting")
ax3.legend(loc=4)

plt.show()